In [23]:
import dlib

In [ ]:
# get the pre-trained face detection model (need to run only once)
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

In [25]:
# prep face detector objects
datFile = "/content/shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector() # locates faces
lm_extractor = dlib.shape_predictor(datFile) # computes landmarks

In [ ]:
import cv2
from google.colab.patches import cv2_imshow # for Colab as imshow crashes there
#########################################################
 # function to draw landamrks
def annotate_img(sample, landmarks):

    landmark_list = []
    for i in range(68):  # Assuming there are 68 landmarks
        landmark = landmarks.part(i)
        x, y = landmark.x, landmark.y
        landmark_list.append((x, y))
        colour = (0, 255, 0) # Default colour
        if i in [21, 22, 39, 42, 33, 51, 57, 48, 54]:
          colour = (0, 0, 255) #Change colour for specific features
          cv2.circle(sample, (x,y), 5, colour, -1)
        else:
          cv2.circle(sample, (x, y), 5, colour, -1)
  
    return sample

#########################################################
file_path = '/content/S055_001_01064616.png'
label = 'surprise' # should change according to emotion in image
sample = cv2.imread(file_path) 
resized_img = cv2.resize(sample, (720, 720))

# any pre-processing (optional)

# locate face and then landmarks for each face
faces = face_detector(sample) # return a list of faces rectangles if any
for face in faces:
  f_vector = []
  landmarks = lm_extractor(sample, face) # returns object of 68 tuple points (x,y)  

  for landmark in landmarks.parts():
    x, y = landmark.x, landmark.y
    f_vector.append(x)
    f_vector.append(y)
  
  f_vector = [label] + f_vector # append emotion label
  print(f_vector)  
#########################################################
  annotated_sample = annotate_img(sample, landmarks) # to draw landmarks
  cv2_imshow(annotated_sample) # comment out when looping over dataset!
#########################################################

f_vector.append(label)
with open('output.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(f_vector)

In [ ]:
import zipfile

#specify the path
zip_file_path = '/content/Testing.zip'

#Extract the ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall('/content/FacialData')

In [ ]:
import os
import cv2
import csv

from google.colab.patches import cv2_imshow # for Colab as imshow crashes there

#########################################################
 # function to draw landmarks
def annotate_img(sample, landmarks):

    landmark_list = []
    for i in range(68):  # Assuming there are 68 landmarks
        landmark = landmarks.part(i)
        x, y = landmark.x, landmark.y
        landmark_list.append((x, y))
        cv2.circle(sample, (x, y), 1, (0, 255, 0), -1)
  
    return sample

#########################################################

#Path to the root folder containing the cartoon character folders
root_folder = '/content/FacialData/Training'

#List of emotions to label each image
emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
#Iterate over each of the character folders
for character_folder in os.listdir(root_folder):
  character_path = os.path.join(root_folder, character_folder)

  #Iterate over each folder pertaining to an emotion
  for emotion in emotions:
    emotion_path = os.path.join(character_path, emotion)

    #Iterate over each image in the emotion folder
    for image_file in os.listdir(emotion_path):
      image_path = os.path.join(emotion_path, image_file)

      #Process the image here and display (Not for large datasets)
      sample = cv2.imread(image_path)
      #resize the image here if need be using the cv2.resize() function

      #We can do further processing here such as landmark extraction and annotations.

      # locate face and then landmarks for each face
      faces = face_detector(sample) # return a list of faces rectangles if any
      for face in faces:
        f_vector = []
        landmarks = lm_extractor(sample, face) # returns object of 68 tuple points (x,y)  

        for landmark in landmarks.parts():
          x, y = landmark.x, landmark.y
          f_vector.append(x)
          f_vector.append(y)
        
        #f_vector = [emotion] + f_vector # append emotion label
        #print(f_vector)  
      #########################################################
        #annotated_sample = annotate_img(sample, landmarks) # to draw landmarks
        #cv2_imshow(annotated_sample) # comment out when looping over dataset!
      #########################################################

        #Save results to a .csv file
        f_vector = [emotion] + f_vector #append the emotion label
        #f_vector.append(emotion)

      csv_path = '/content/all_annotations.csv'

      with open(csv_path, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(f_vector)

In [41]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UserWarning)

In [104]:
#Code snipped to make a .csv file with the necessary 'emotion' header

data = pd.read_csv('extra_annotations_features.csv', header=None)
data.rename(columns={0: 'emotion'}, inplace = True)
data.to_csv('extra_annotations_features_label.csv', index=False) #save new files with header

#num_columns = len(data.columns)

#column_names = ['emotion'] + [f'feature({i})' for i in range(1, num_columns)]

#data.columns = column_names

#data.to_csv('data_with_column_name.csv', index=False)

In [ ]:
#TEST THE SVM MODEL ON NEW DATA - VALIDATION STAGE/TESTING DATA
import pandas as pd
import cv2
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics

def preprocess_and_extract_features(new_image):

#Perform preprocessing steps here (normalise/resizing)

#Extract the landmark features from the image using the provided landmarks

    # locate face and then landmarks for each face
  faces = face_detector(new_image) # return a list of faces rectangles if any
  landmark_features = [] #List to store landmark features for all faces

  for face in faces:
    f_vector = []
    landmarks = lm_extractor(new_image, face) # returns object of 68 tuple points (x,y)  

    for landmark in landmarks.parts():
      x, y = landmark.x, landmark.y
      f_vector.append(x)
      f_vector.append(y)

    landmark_features.append(f_vector)
    
  return landmark_features

########################################################
#Read .csv to a dataframe

#data = pd.read_csv('all_annotations_new.csv')
########################################################
#Separate the features from the labels
y_str = data['emotion']
x = data.drop('emotion', axis=1)
########################################################
# encode the labels, useful when the model cannot process categorical data
y = preprocessing.LabelEncoder().fit_transform(list(y_str)) # labels as numbers
########################################################
#Split data and train a model
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state = 0)

#Scaling the input data
scaler = StandardScaler()

#Fit the scaler on the training data
scaler.fit(x_train)

#Scale the training and testing data
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# create and train a linear model
clf = SVC(kernel='linear', C=1)
clf.fit(x_train, y_train)

# predict on test data
y_pred = clf.predict(x_test)
#print("accuracy:", metrics.accuracy_score(y_test, y_pred))

########################################################
#Test on new inputs
# load a new image

#List to store the image paths
image_paths = []
root_folder = "/content/drive/MyDrive/FacialData/Validation"

#List of emotions to label each image
emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

#Iterate through each character folder
for character_folder in os.listdir(root_folder):
  character_path = os.path.join(root_folder, character_folder)

  #Iterate over each folder pertaining to an emotion
  for emotion in emotions:
    emotion_path = os.path.join(character_path, emotion)

    #Iterate over each image in the emotion folder
    for image_file in os.listdir(emotion_path):
      image_path = os.path.join(emotion_path, image_file)
      image_paths.append(image_path)

#Select a random subset of image paths
num_images = min(len(image_paths), 3790) # number of paths to select
random_image_path = random.sample(image_paths, num_images)

#iterate over the randomly selected image paths
correct_predictions = 0
total_predictions = 0
no_predictions = 0

for image_path in random_image_path:
  new_image = cv2.imread(image_path)
  
  if new_image is not None:

    # extract its features
    # predict on the extracted features
    new_features = preprocess_and_extract_features(new_image)

    #Check if features were found and extracted
    if len(new_features) == 0:
      #print("No features found in this image: ", image_path)
      no_predictions += 1
      continue

    new_features = np.reshape(new_features, (1, -1))
    new_prediction = clf.predict(new_features)

    #Extract the emotion label from the image file path
    emotion = os.path.basename(os.path.dirname(image_path))

    #Check if the predicted label matches the ground truth label
    if emotion in emotions and new_prediction[0] == emotions.index(emotion):
      correct_predictions += 1

    total_predictions += 1

#Calculate the accuracy here
print("Total predictions", total_predictions)
print("Correct Predictions" , correct_predictions)
accuracy = (correct_predictions / (total_predictions + no_predictions) * 100)
print("Accuracy: ", accuracy)
print("Images with no found features: ", no_predictions)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
########################################################
# function to plot points
def plot_2features(x1, x2, y_str, y):
  # for plotting with custom legend labels
  unique = np.unique(y_str)
  u_code = np.unique(y)
  colors = [plt.cm.jet(i/float(len(unique)-1)) for i in range(len(unique))]
  for i, u in enumerate(u_code):
      xi = x1[y == u]
      yi = x2[y == u]
      plt.scatter(xi, yi, color=colors[u], label=unique[u])
  pl = plt.legend()
  plt.title('Scatter of two features')
########################################################
# function to plot decision plane
def visualisation(x1, x2, data, y_str, y):
  # to visualise decision boundry (from sklearn tutorial)
#https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html  
  # create a mesh to plot in
  h = 1  # step size in the mesh
  x_min, x_max = x1.min() - 1, x1.max() + 1
  y_min, y_max = x2.min() - 1, x2.max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                      np.arange(y_min, y_max, h))
  # classify the mesh points
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
  # Put the result into a color plot
  Z = Z.reshape(xx.shape)
  plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.2, linewidths=0.5)
  plt.contour(xx, yy, Z, colors='k', levels=[0], linewidths=2.0)  # Adjust linewidths parameter
  # Plot also the training points
  #plt.scatter(x1, x2, c=y, cmap=plt.cm.coolwarm)
  plot_2features(x1, x2, y_str, y)
  plt.xlabel(data.columns[f1])
  plt.ylabel(data.columns[f2])
  plt.title('SVC Decision Boundary (2 Features)')

  plt.show()
########################################################
# TODO(1): repeat what you did for the previous code cell
# read data into dataframe

#data = pd.read_csv('annotations_features_train.csv')
data = pd.read_csv('all_annotations_new.csv')
########################################################
# drop all rows except for two classes
# you can change to view features for any two classes
data = data[(data.emotion == 'joy')| (data['emotion'] == 'anger')]
########################################################
# separate the features from the labels
y_str = data['emotion']
x = data.drop('emotion', axis=1)
########################################################
y = preprocessing.LabelEncoder().fit_transform(list(y_str)) # labels as numbers
########################################################
# for visualisation, choose any two features
# TODO(2): change which two feature are used and observe graphs
f1 = 48
f2 = 54
x1 = x.iloc[:, f1]
x2 = x.iloc[:, f2]
x  = np.column_stack((x1, x2))
########################################################
# TODO(3): change model parameters and observe decision function
clf = SVC(kernel='linear', C=10.0)
clf.fit(x, y)  
########################################################
visualisation(x1, x2, data, y_str, y)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.preprocessing import LabelEncoder

# Read the data from the CSV file
#data = pd.read_csv('annotations_features_train.csv')
data = pd.read_csv('all_annotations_new.csv')

# Separate the features from the labels
y_str = data['emotion']
x = data.drop('emotion', axis=1)

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_str)

# Select two features for plotting the decision boundary
x_subset = x.iloc[:, [4, 1]].copy()

# Fit the model and get the separating hyperplane
clf = svm.SVC(kernel="linear", C=1.0)
clf.fit(x_subset, y)

# Plot the samples
scatter = plt.scatter(x_subset.iloc[:, 0], x_subset.iloc[:, 1], c=y, cmap=plt.cm.Paired, edgecolors="k")

# Plot the decision boundary
ax = plt.gca()
disp = DecisionBoundaryDisplay.from_estimator(
    clf,
    x_subset.values,
    plot_method="contour",
    colors="k",
    levels=[0],
    alpha=0.5,
    linestyles=["-"],
    ax=ax,
)

# Get the unique emotions and their corresponding labels
unique_emotions = label_encoder.inverse_transform(np.unique(y))
handles = [plt.Line2D([], [], marker="o", linestyle="", color=scatter.cmap(scatter.norm(label_encoder.transform([emotion]))[0])) for emotion in unique_emotions]

# Add the legend
plt.legend(handles, unique_emotions, loc="upper right")
plt.xlabel('LeftEyebrow')
plt.ylabel('RightEyebrow')
plt.show()


In [ ]:
#CODE CELL THAT WILL WRITE THE CSV FILE USING THE TRAINING DATASET
import pandas as pd
import cv2
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from google.colab.patches import cv2_imshow
import math
import csv

#########################################################
def dist(p1, p2): # Euclidean distance 
    x1 = p1.x
    y1 = p1.y
    x2 = p2.x
    y2 = p2.y  
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

#########################################################
# Function to draw landmarks in different colours so we can see what we are targetting
def annotate_img(sample, landmarks):
    landmark_list = []
    for i in range(68):
        landmark = landmarks.part(i)
        x, y = landmark.x, landmark.y
        landmark_list.append((x, y))
        colour = (0, 255, 0)  # Default color
        if i in [18, 19, 20, 21, 39, 22, 23, 24, 25, 42, 33, 48, 49, 50, 51, 52, 53, 54, 36, 37, 38, 49, 41, 43, 44, 45, 46, 47]:
            colour = (0, 0, 255)  # Change colour for specific features
        cv2.circle(sample, (x, y), 3, colour, -1)
    return sample

#########################################################
# Function to extract the RIGHT EYE feature
def extract_righteye_feature(landmarks):
    right_eye_points = [43, 44, 45, 46, 47]  # Right eye landmarks excluding point 39
    right_inner_eye_point = 42  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eye landmarks and inner eye point
    dist_sum = 0
    for point in right_eye_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_inner_eye_point)
        distance = dist(p1, p2)
        distance_42_43 = dist(landmarks.part(42), landmarks.part(43))
        normalised_distance = distance / distance_42_43
        dist_sum += normalised_distance

    return dist_sum

#########################################################
# Function to extract the LEFT EYE feature
def extract_lefteye_feature(landmarks):
    left_eye_points = [36, 37, 38, 49, 41]  # Left eye landmarks excluding point 39
    left_inner_eye_point = 39  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eye landmarks and inner eye point
    dist_sum = 0
    for point in left_eye_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_inner_eye_point)
        distance = dist(p1, p2)
        distance_39_38 = dist(landmarks.part(39), landmarks.part(38))
        normalised_distance = distance / distance_39_38
        dist_sum += normalised_distance

    return dist_sum

#########################################################
# Function to extract the LEFT EYEBROW feature
def extract_left_eyebrow_feature(landmarks):
    left_eyebrow_points = [18, 19, 20, 21]  # Left eyebrow landmarks excluding point 39
    left_inner_eye_point = 39  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eyebrow landmarks and inner eye point
    dist_sum = 0
    for point in left_eyebrow_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_inner_eye_point)
        distance = dist(p1, p2)
        distance_39_21 = dist(landmarks.part(39), landmarks.part(21))
        normalised_distance = distance / distance_39_21
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract the RIGHT EYEBROW
def extract_right_eyebrow_feature(landmarks):
    right_eyebrow_points = [22, 23, 24, 25]  # right eyebrow landmarks excluding point 39
    right_inner_eye_point = 42  # Point to which distances are normalised

    # Calculate the sum of normalised distances between right eyebrow landmarks and inner eye point
    dist_sum = 0
    for point in right_eyebrow_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_inner_eye_point)
        distance = dist(p1, p2)
        distance_42_25 = dist(landmarks.part(42), landmarks.part(25))
        normalised_distance = distance / distance_42_25
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract LEFT LIP
def extract_left_lip_feature(landmarks):  
    left_lip_points = [48, 49, 50, 51]  #Left lip landmarks excluding point 34
    left_stationary_point = 33    #Point to which distances are normalised

          # Calculate the sum of normalised distances between left lip landmarks and the stationary lip point
    dist_sum = 0
    for point in left_lip_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_stationary_point)
        distance = dist(p1, p2)
        distance_33_51 = dist(landmarks.part(33), landmarks.part(51))
        normalised_distance = distance / distance_33_51
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract RIGHT_LIP
def extract_right_lip_feature(landmarks):
    right_lip_points = [52, 53, 54, 51]  #Right lip landmarks excluding point 34
    right_stationary_point = 33    #Point to which distances are normalised

          # Calculate the sum of normalised distances between right lip landmarks and stationary lip point
    dist_sum = 0
    for point in right_lip_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_stationary_point)
        distance = dist(p1, p2)
        distance_33_51 = dist(landmarks.part(33), landmarks.part(51))
        normalised_distance = distance / distance_33_51
        dist_sum += normalised_distance

    return dist_sum  

#########################################################
#Function for the LIP WIDTH
#Get the distance between edges of lips and normalise
def extract_lip_width_feature(landmarks):

    dist_sum = 0
    distance_width = dist(landmarks.part(48), landmarks.part(54))
    distance_normal_points = dist(landmarks.part(33), landmarks.part(51))
    normalised_distance = distance_width / distance_normal_points
    dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function for the LIP HEIGHT
#Distance between the upper and lower lip and normalise
def extract_lip_height_feature(landmarks):

    dist_sum = 0
    distance_width = dist(landmarks.part(51), landmarks.part(57))
    distance_normal_points = dist(landmarks.part(33), landmarks.part(51))
    normalised_distance = distance_width / distance_normal_points
    dist_sum += normalised_distance

    return dist_sum
#########################################################
# Function to compute distances and write to CSV
def compute_distances_and_write_csv(file_path):
    label = os.path.basename(os.path.dirname(file_path))  # Change the label according to the emotion in the image
    sample = cv2.imread(file_path)
    resized_img = cv2.resize(sample, (720, 720))

    # Locate face and then landmarks for each face
    faces = face_detector(sample)  # Return a list of face rectangles if any

    for face in faces:
        f_vector = []
        landmarks = lm_extractor(sample, face)  # Returns object of 68 tuple points (x, y)

        # Extract left eyebrow feature
        left_eye_feature = extract_lefteye_feature(landmarks)
        right_eye_feature = extract_righteye_feature(landmarks)
        left_eyebrow_feature = extract_left_eyebrow_feature(landmarks)
        right_eyebrow_feature = extract_right_eyebrow_feature(landmarks)
        left_lip_feature = extract_left_lip_feature(landmarks)
        right_lip_feature = extract_right_lip_feature(landmarks)
        lip_width_feature = extract_lip_width_feature(landmarks)
        lip_height_feature = extract_lip_height_feature(landmarks)

        #Append the emotion label and then the features
        f_vector.append(label)
        f_vector.append(left_eye_feature)
        f_vector.append(right_eye_feature)
        f_vector.append(left_eyebrow_feature)
        f_vector.append(right_eyebrow_feature)
        f_vector.append(left_lip_feature)
        f_vector.append(right_lip_feature)
        f_vector.append(lip_width_feature)
        f_vector.append(lip_height_feature)

        # Print the feature vector
        print(f_vector)

        # Annotate the sample image with landmarks
        annotated_sample = annotate_img(sample, landmarks)
        #cv2_imshow(annotated_sample)  # Comment out when looping over the dataset!

        # Write the feature vector to CSV
        with open('extra_annotations_features.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(f_vector)
################################################################
#List to store the image paths
image_paths = []
root_folder = "/content/drive/MyDrive/FacialData/Training"

#List of emotions to label each image
emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

#Iterate through each character folder
for character_folder in os.listdir(root_folder):
  character_path = os.path.join(root_folder, character_folder)

  #Iterate over each folder pertaining to an emotion
  for emotion in emotions:
    emotion_path = os.path.join(character_path, emotion)

    #Iterate over each image in the emotion folder
    for image_file in os.listdir(emotion_path):
      image_path = os.path.join(emotion_path, image_file)
      image_paths.append(image_path)


      # Call the function with the file path of your test image
      compute_distances_and_write_csv(image_path)


In [ ]:
#CODE CELL TO TEST THE MODEL ON THE VALIDATION DATA USING NEW CSV

import pandas as pd
import cv2
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from google.colab.patches import cv2_imshow
import math
import csv

#########################################################
def dist(p1, p2): # Euclidean distance 
    x1 = p1.x
    y1 = p1.y
    x2 = p2.x
    y2 = p2.y  
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

#########################################################
# Function to draw landmarks in different colours so we can see what we are targetting
def annotate_img(sample, landmarks):
    landmark_list = []
    for i in range(68):
        landmark = landmarks.part(i)
        x, y = landmark.x, landmark.y
        landmark_list.append((x, y))
        colour = (0, 255, 0)  # Default color
        if i in [18, 19, 20, 21, 39, 22, 23, 24, 25, 42, 33, 48, 49, 50, 51, 52, 53, 54, 36, 37, 38, 49, 41, 43, 44, 45, 46, 47]:
            colour = (0, 0, 255)  # Change colour for specific features
        cv2.circle(sample, (x, y), 3, colour, -1)
    return sample

#########################################################
# Function to extract the RIGHT EYE feature
def extract_righteye_feature(landmarks):
    right_eye_points = [43, 44, 45, 46, 47]  # Right eye landmarks excluding point 39
    right_inner_eye_point = 42  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eye landmarks and inner eye point
    dist_sum = 0
    for point in right_eye_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_inner_eye_point)
        distance = dist(p1, p2)
        distance_42_43 = dist(landmarks.part(42), landmarks.part(43))
        normalised_distance = distance / distance_42_43
        dist_sum += normalised_distance

    return dist_sum

#########################################################
# Function to extract the LEFT EYE feature
def extract_lefteye_feature(landmarks):
    left_eye_points = [36, 37, 38, 49, 41]  # Left eye landmarks excluding point 39
    left_inner_eye_point = 39  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eye landmarks and inner eye point
    dist_sum = 0
    for point in left_eye_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_inner_eye_point)
        distance = dist(p1, p2)
        distance_39_38 = dist(landmarks.part(39), landmarks.part(38))
        normalised_distance = distance / distance_39_38
        dist_sum += normalised_distance

    return dist_sum

#########################################################
# Function to extract the LEFT EYEBROW feature
def extract_left_eyebrow_feature(landmarks):
    left_eyebrow_points = [18, 19, 20, 21]  # Left eyebrow landmarks excluding point 40
    left_inner_eye_point = 39  # Point to which distances are normalized

    # Calculate the sum of normalized distances between left eyebrow landmarks and inner eye point
    dist_sum = 0
    for point in left_eyebrow_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_inner_eye_point)
        distance = dist(p1, p2)
        distance_39_21 = dist(landmarks.part(39), landmarks.part(21))
        normalised_distance = distance / distance_39_21
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract the RIGHT EYEBROW
def extract_right_eyebrow_feature(landmarks):
    right_eyebrow_points = [22, 23, 24, 25]  # right eyebrow landmarks excluding point 40
    right_inner_eye_point = 42  # Point to which distances are normalised

    # Calculate the sum of normalised distances between right eyebrow landmarks and inner eye point
    dist_sum = 0
    for point in right_eyebrow_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_inner_eye_point)
        distance = dist(p1, p2)
        distance_42_25 = dist(landmarks.part(42), landmarks.part(25))
        normalised_distance = distance / distance_42_25
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract LEFT LIP
def extract_left_lip_feature(landmarks):  
    left_lip_points = [48, 49, 50, 51]  #Left lip landmarks excluding point 34
    left_stationary_point = 33    #Point to which distances are normalised

          # Calculate the sum of normalised distances between left lip landmarks and the stationary lip point
    dist_sum = 0
    for point in left_lip_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(left_stationary_point)
        distance = dist(p1, p2)
        distance_33_51 = dist(landmarks.part(33), landmarks.part(51))
        normalised_distance = distance / distance_33_51
        dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function to extract RIGHT_LIP
def extract_right_lip_feature(landmarks):
    right_lip_points = [52, 53, 54, 51]  #Right lip landmarks excluding point 34
    right_stationary_point = 33    #Point to which distances are normalised

          # Calculate the sum of normalised distances between right lip landmarks and stationary lip point
    dist_sum = 0
    for point in right_lip_points:
        p1 = landmarks.part(point)
        p2 = landmarks.part(right_stationary_point)
        distance = dist(p1, p2)
        distance_33_51 = dist(landmarks.part(33), landmarks.part(51))
        normalised_distance = distance / distance_33_51
        dist_sum += normalised_distance

    return dist_sum  

#########################################################
#Function for the LIP WIDTH
#Get the distance between edges of lips and normalise
def extract_lip_width_feature(landmarks):

    dist_sum = 0
    distance_width = dist(landmarks.part(48), landmarks.part(54))
    distance_normal_points = dist(landmarks.part(33), landmarks.part(51))
    normalised_distance = distance_width / distance_normal_points
    dist_sum += normalised_distance

    return dist_sum

#########################################################
#Function for the LIP HEIGHT
#Distance between the upper and lower lip and normalise
def extract_lip_height_feature(landmarks):

    dist_sum = 0
    distance_width = dist(landmarks.part(51), landmarks.part(57))
    distance_normal_points = dist(landmarks.part(33), landmarks.part(51))
    normalised_distance = distance_width / distance_normal_points
    dist_sum += normalised_distance

    return dist_sum
#########################################################

def preprocess_and_extract_features(new_image):
#Perform preprocessing steps here (normalise/resizing)

#Extract the landmark features from the image using the provided landmarks
   # Locate face and then landmarks for each face
    faces = face_detector(new_image)  # Return a list of face rectangles if any

    for face in faces:
        landmark_features = []
        landmarks = lm_extractor(new_image, face)  # Returns object of 68 tuple points (x, y)

        # Extract left eyebrow feature
        left_eye_feature = extract_lefteye_feature(landmarks)
        right_eye_feature = extract_righteye_feature(landmarks)
        left_eyebrow_feature = extract_left_eyebrow_feature(landmarks)
        right_eyebrow_feature = extract_right_eyebrow_feature(landmarks)
        left_lip_feature = extract_left_lip_feature(landmarks)
        right_lip_feature = extract_right_lip_feature(landmarks)
        lip_width_feature = extract_lip_width_feature(landmarks)
        lip_height_feature = extract_lip_height_feature(landmarks)

        #Append the emotion label and then the features
        landmark_features.append(left_eye_feature)
        landmark_features.append(right_eye_feature)
        landmark_features.append(left_eyebrow_feature)
        landmark_features.append(right_eyebrow_feature)
        landmark_features.append(left_lip_feature)
        landmark_features.append(right_lip_feature)
        landmark_features.append(lip_width_feature)
        landmark_features.append(lip_height_feature)

        # Print the feature vector
        #print(landmark_features)

        return landmark_features

########################################################
#Read .csv to a dataframe

data = pd.read_csv('extra_annotations_features_label.csv')
########################################################
#Separate the features from the labels
y_str = data['emotion']
x = data.drop('emotion', axis=1)
########################################################
# encode the labels, useful when the model cannot process categorical data
y = preprocessing.LabelEncoder().fit_transform(list(y_str)) # labels as numbers
########################################################
#Split data and train a model
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state = 0)

#Scaling the input data
scaler = StandardScaler()

#Fit the scaler on the training data
scaler.fit(x_train)

#Scale the training and testing data
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# create and train a linear model
clf = SVC(kernel='linear', C=10)
clf.fit(x_train, y_train)

# predict on test data
y_pred = clf.predict(x_test)
#print("accuracy:", metrics.accuracy_score(y_test, y_pred))

########################################################
#Test on new inputs
# load a new image

#List to store the image paths
image_paths = []
root_folder = "/content/drive/MyDrive/FacialData/Validation"

#List of emotions to label each image
emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
predicted_emotions = []

#Iterate through each character folder
for character_folder in os.listdir(root_folder):
  character_path = os.path.join(root_folder, character_folder)

  #Iterate over each folder pertaining to an emotion
  for emotion in emotions:
    emotion_path = os.path.join(character_path, emotion)

    #Iterate over each image in the emotion folder
    for image_file in os.listdir(emotion_path):
      image_path = os.path.join(emotion_path, image_file)
      image_paths.append(image_path)

#Select a random subset of image paths
num_images = min(len(image_paths), 3790) # number of paths to select
random_image_path = random.sample(image_paths, num_images)

#iterate over the randomly selected image paths
correct_predictions = 0
total_predictions = 0
no_predictions = 0

for image_path in random_image_path:
  new_image = cv2.imread(image_path)
  
  if new_image is not None:

    # extract its features
    # predict on the extracted features
    new_features = preprocess_and_extract_features(new_image)


    #Check if features were found and extracted
    if new_features is None:
      no_predictions += 1
      continue

    if len(new_features) == 0:
      #print("No features found in this image: ", image_path)
      no_predictions += 1
      continue

    new_features = np.reshape(new_features, (1, -1))
    new_prediction = clf.predict(new_features)

    predicted_emotion = emotions[new_prediction[0]]
    predicted_emotions.append(predicted_emotion)

    #Extract the emotion label from the image file path
    emotion = os.path.basename(os.path.dirname(image_path))
    #print("Predicted Emotion: ",)

    #Check if the predicted label matches the ground truth label
    if emotion in emotions and new_prediction[0] == emotions.index(emotion):
      correct_predictions += 1

    total_predictions += 1

#Calculate the accuracy here
print("Total predictions", total_predictions)
print("Correct Predictions" , correct_predictions)
accuracy = (correct_predictions / (total_predictions + no_predictions) * 100)
print("Accuracy: ", accuracy)
print("Images with no found features: ", no_predictions)